# Binaryzacja

### Zadanie domowe - binaryzacja adaptacyjna w oknach z interpolacją.

Pokazana w ramach podstawowej części ćwiczenia binaryzacja adaptacyjna działa dobrze, ale jest dość złożona obliczeniowo (choć oczywiście należy mieć świadomość, że implementację można zoptymalizować i wyeliminować pewne powtarzające się obliczenia).
Zbliżone rozwiązanie można również realizować w nieco innym wariancie - w oknach.
Ogólna idea jest następująca: wejściowy obraz dzielimy na nienachodzące (rozłączne) okna - wygodnie jest założyć, że są one kwadratowe i o rozmiarze będącym potęgą liczby 2.
W każdym z okien obliczamy próg - niech to będzie średnia i stosujemy do binaryzacji lokalnej.
Jak nietrudno się domyślić efekt nie będzie dobry, gdyż na granicach okien wystąpią artefakty.
Aby je wyeliminować należy zastosować interpolację, co zostanie szczegółowo opisane poniżej.
Warto zaznaczyć, że podobny mechanizm interpolacji stosowany jest w poznanym wcześniej algorytmie CLAHE.
Zainteresowane osoby odsyłam do artykułu na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization) oraz do artykułu o metodzie CLAHE - Zuiderveld, Karel. “Contrast Limited Adaptive Histograph Equalization.” Graphic Gems IV. San Diego: Academic Press Professional, 1994. 474–485.



Na początek zaimplementujemy wariant metody bez interpolacji:
1. Wczytaj obraz _rice.png_.
2. W dwóch pętlach `for`, dla okien o ustalonym wymiarze $W$ (potęga 2), oblicz średnią:
- pętle powinny mieć krok $W$,
- wynik (tj. średnie) należy zapisać w pomocniczej tablicy,
- przydatny operator to `//` - dzielenie całkowitoliczbowe (*floor division*).

3. W kolejnych dwóch pętlach `for` (tym razem o kroku 1) przeprowadź binaryzację z wyznaczonymi progami.
   Tu oczywiście należy się sprytnie odwołać do wyników z tablicy pomocniczej.
   Wyświetl wyniki - czy jest on poprawny?
   Podpowiedź - błędy lepiej widać dla mniejszego rozmiaru okna (np. 16 x 16).

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

#wczytanie obrazu
if not os.path.exists("rice.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/rice.png --no-check-certificate
        
rice = cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE)   


(X,Y) = rice.shape
W=16
obraz_binaryzacja = np.zeros(rice.shape)
progi=np.zeros((X//W,Y//W)) #dzielenie całkowitoliczbowe

for i in range(0, 256,W):
    for j in range (0, 256,W):
        tablica_pom = rice[i:i+W,j:j+W] #tablica pomocnicza 
        progi[i//W][j//W] = np.mean(tablica_pom)

        
for i in range(X):
    for j in range(Y):
        obraz_binaryzacja[i][j]=(rice[i][j]>progi[i//W][j//W])

        
plt.gray()
plt.imshow(rice)
plt.xticks([])
plt.yticks([])
plt.title("obraz")
plt.show()

#Wywołanie funkcji
plt.gray()
plt.imshow(obraz_binaryzacja)
plt.axis("off")
plt.title("obraz bez interpolacji")
plt.show()

4. Rozwiązaniem problemu artefaktów na obrazie jest zastosowanie interpolacji.
   Próg binaryzacji dla danego okna wyliczany jest na podstawie progów z sąsiednich okien.
   ![Ilustracja koncepcji interpolacji](https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/clahe_tile_interpolation.png)

   Koncepcja została przedstawiona na powyższym rysunku.
   Możliwe są 3 przypadki:
   - piksel leży w rogach obrazu (kolor czerwony) - wtedy za próg przyjmuje się wartość średniej obliczonej dla danego okna,
   - piksel leży na krawędzi obrazu (kolor zielony) - wtedy za próg przyjmuje się wartość obliczoną na podstawie średnich z dwóch sąsiednich okien,
   - piksel leży w środku (kolor fioletowy) - wtedy próg jest obliczany na podstawie 4 sąsiednich okien.

   Uwaga. Proszę zwrócić uwagę, że sprawa jest dość złożona.
   Obraz dzielimy na okna (dla nich liczymy średnią) i następnie każde z okien "wirtualnie" na cztery sub-okna (linie przerywane).
   To ułatwia znalezienie środków okien (czarne kwadraty), które są wykorzystywane w interpolacji.

5. Implementujemy interpolację.
   Potrzebujemy do tego znać progi (jeden, dwa lub cztery), ale dla przejrzystości obliczeń lepiej zawsze przyjąć cztery oraz odległości od rozważnego piksela do środka sąsiednich okien (też w ogólnym przypadku 4):
   - całość sprowadza się do określania pozycji piksela,
   - na początek rozważmy przypadek czterech narożników (kolor czerwony na rysunku) - trzeba napisać `if`, który je wyznaczy,
   - warto sprawdzić, czy nie popełniliśmy błędu i np. tymczasowo do obrazu wynikowego w tym miejscu przypisać wartość 255. Efekt powinien być taki, że widoczne będą tylko narożniki.
   - drugi przypadek do brzegi (kolor zielony) - postępujemy podobnie jak przy narożnikach, przy czym osobno wydzielamy brzegi pionowe i poziome. Tu też warto sobie obrazek "pokolorować".
   - na koniec wyznaczamy piksele w środku.
   - analizując poprawność proszę zwrócić uwagę na to, żeby nie było przerw pomiędzy obszarami.
   - mając podział możemy dla każdego z obszarów wyliczyć cztery progi ($t11, t12, t21, t22$):
        - dla narożników wartość ta będzie identyczna i wynosi po prostu `t11 =t[jT][iT]`, gdzie `iT=i//W` oraz `jT=j//W`.
          Uwaga. Proszę używać indeksów tymczasowych $jT,iT$, gdyż będą potrzebne w dalszych obliczeniach.
        - dla brzegów pionowych występują dwie wartości: okno bieżące i sąsiednie.
          Wyznaczenie współrzędnej poziomej jest proste (jak dla narożników).
          Nad współrzędną pionową trzeba się chwilę zastanowić - aby nie rozważać wielu przypadków można od bieżącej współrzędnej odjąć połowę rozmiaru okna i dopiero później wykonać dzielenie przez rozmiar okna.
          W ten sposób otrzymujemy indeks okna o mniejszej współrzędnej.
          Indeks drugiego uzyskamy dodając 1.
          Proszę się zastanowić dlaczego to działa - najlepiej to sobie rozrysować.
        - dla brzegów poziomych należy postąpić analogicznie,
        - obliczenia dla obszaru wewnątrz powinny być już oczywiste.
   - kolejny krok to wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami.
     Przykładowo dla osi X wygląda to następująco: `dX1 = i - W/2 - iT*W` oraz `dX2 = (iT+1)*W - i-W/2`.
     Dla osi Y analogicznie.
     Ponownie proszę się zastanowić dlaczego to jest poprawne - najlepiej to sobie narysować.
   - ostatni krok to interpolacja dwuliniowa.
     Wykonamy ją w trzech krokach:
     - interpolacja w osi X dla dwóch górnych okien - sprowadza się ona do średniej ważonej pomiędzy wartościami $t11$ i $t12$, przy czym wagi to odpowiednio $dX2/W$ i $dX1/W$.
       Ponownie na podstawie rysunku proszę to przemyśleć.
     - interpolacja w osi X dla dolnych okien jest analogiczna,
     - interpolacja w osi Y również jest analogiczna, z tym, że wejściem są dwa wyniki interpolacji w poziomie.

6. "Kropka nad i" to oczywiście binaryzacja z wyznaczonym poprzez interpolację progiem - proszę dobrać rozmiar okna.
7. Na koniec proszę porównać na wspólnym rysunku poznane metody binaryzacji:
- Otsu,
- lokalna na podstawie średniej,
- lokalna Sauvoli,
- lokalna w oknach bez interpolacji,
- lokalna w oknach z interpolacją.

Proszę pod porównaniem, w osobnej sekcji *markdown*, krótko skomentować uzyskane wyniki.

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

#wczytanie obrazu
if not os.path.exists("rice.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/rice.png --no-check-certificate
        
rice = cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE) 


def interpolacja(obraz,potega):
    W = 2**potega
    tablica = [[]] #tablica dwuwymiarowa do okreslenia naroznikow iT i jT
    for i in range(0,256,W):
        for j in range(0,256,W):
            tablica[i//W].append(obraz[i:i+W,j:j+W].mean()) #srednia
        tablica.append([])
    tablica.pop(-1) #zdjecie z tablicy ostatniego elementu
    new_obraz = np.zeros(obraz.shape)
    tablica = np.array(tablica)
    

    for i in range(obraz.shape[0]):
        for j in range(obraz.shape[0]):
            if (((i < W//2 + 1) and (j < W//2 + 1)) or\
           (i > 256 - W//2 - 1) and (j > 256 - W//2 - 1)) or\
           ((i > 256 - 1 - W//2) and (j < W//2 + 1)) or\
           ((i < W//2 + 1) and (j > 256 - 1 - W//2)):
                  

                
                jT = j//W  #narozniki 
                iT = i//W  #narozniki 
                new_obraz[i,j] = tablica[iT,jT] < obraz[i,j]  ###########if wyznaczenie czterech nowych naroznikow(kolor czerwony)
            #brzegi pionowe( od bieżącej współrzędnej odejmujemy połowę rozmiaru okna i dopiero później wykonujemy dzielenie przez rozmiar okna)
            #indeks okna o mniejszej współrzędnej ,   Indeks drugiego uzyskamy dodając 1
            elif (j > 256 - W//2 - 1) or ((j < W//2 + 1)):
                iT = (i-W//2)//W
                dY1 = i - W/2 - iT*W
                dY2 = (iT+1)*W - (i - W/2) 

                #wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami dla osi Y

                new_obraz[i,j] = int(int(dY1+1)/int(W)*tablica[iT,jT]+int(dY2)/int(W)*tablica[iT,jT]) < obraz[i,j]
            
            #analogicznie to samo co wyzej
             #brzegi poziome
            elif (i > 256 - 1 - W//2) or (i < W//2 + 1):  
                #wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami dla osi X

                jT = (j-W//2)//W
                dX1 = j - W/2 - jT*W
                dX2 = (jT+1)*W - (j - W/2)
                new_obraz[i,j] = int(int(dX1)/int(W)*tablica[iT,jT+1]+int(dX2)/int(W)*tablica[iT,jT]) < obraz[i,j]
            
            else: #warunek na ramke wyznaczenie pikseli na środku
                
                #wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami dla osi Y
                dY1 = i - W/2 - iT*W
                dY2 = (iT+1)*W - (i - W/2) 
                iT = (i-W//2)//W
                #wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami dla osi X
                dX1 = j - W/2 - jT*W
                dX2 = (jT+1)*W - (j - W/2) 
                jT = (j-W//2)//W
                new_obraz[i,j] = int(int(dX1)/int(W)*int(dY1)/int(W)*tablica[iT+1,jT+1]+\
                                     int(dX2)/int(W)*int(dY2)/int(W)*tablica[iT,jT] +\
                                     int(dX1)/int(W)*int(dY2)/int(W)*tablica[iT,jT+1]+\
                                     int(dX2)/int(W)*int(dY1)/int(W)*tablica[iT+1,jT]) < obraz[i,j]





    return new_obraz
k = 4
plt.gray()
plt.imshow(interpolacja(rice,k))
plt.title("z interpolacja")
plt.show()

In [ ]:
#Metoda Otsu
Otsu=cv2.threshold(rice,127,maxval=255,type=cv2.THRESH_OTSU)

#plt.imshow(Otsu[1],'gray')




#Binaryzacja lokalna na podstawie średniej
def binary(obraz, W):
    (X, Y) = obraz.shape
    bin_obraz = np.zeros(obraz.shape)
   
    for j in range(int(W/2), Y-int(W/2)):
        for i in range (int(W/2), X-int(W/2)):
            otoczenie = obraz[i-int(W/2):i+int(W/2),j-int(W/2):j+int(W/2)]
            srednia = np.mean(otoczenie)
            bin_obraz[i][j] = obraz[i][j] > srednia

    return bin_obraz


#plt.imshow(bin_obraz, 'gray')




#bez interpolacji
#plt.imshow(obraz_binaryzacja)

#z interpolacja
#plt.imshow(interpolacja(rice,k))


In [ ]:
f, (a1,a2,a3,a4,a5) = plt.subplots(1, 5, figsize=(15,15))
a1.imshow(rice)
a2.imshow(Otsu[1],'gray')
a3.imshow(binary(rice, 10) )
a4.imshow(obraz_binaryzacja)
a5.imshow(interpolacja(rice,k))
#a5.imshow(binaryzacja(rice,120)) 

a1.set_title("Oryginał")
a2.set_title('OTSU')
a3.set_title('Na podstawie średniej')
a4.set_title('Bez interpolacji')
a5.set_title('Z interpolacja')


a1.set_xticks([])
a1.set_yticks([])
a2.set_xticks([])
a2.set_yticks([])
a3.set_xticks([])
a3.set_yticks([])
a4.set_xticks([])
a4.set_yticks([])
a5.set_xticks([])
a5.set_yticks([])


**Wnioski**
Jeżeli chodzi o najlepszą binaryzacje to zdecydowanie wygrywa binaryzacja lokalna na podstawie średniej, lecz jest ona najbardziej czasochłonna. Jeżeli chodzi o szybkość to wygrywa binaryzacja Otsu. Zawiodłem się na binaryzacji z interpolacją ponieważ nie można ukrywać, że jest ona efektywniejsza od binaryzacji bez interpolacji, jednak jeżeli chodzi o sam algorytm to dla tej pierwszej jest on bardziej skomplikowany. 